# Imports

In [1]:
"""
Main entry point for the Enhanced PowerBI Agent.
"""

import warnings
import os

# Suppress all warnings at the beginning of the script
warnings.filterwarnings("ignore")

# Suppress urllib3 warnings
os.environ["PYTHONWARNINGS"] = "ignore:Unverified HTTPS request"

# Import libraries after suppressing warnings
from langchain.agents import AgentExecutor, ZeroShotAgent, Tool
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

from custom_llm import CustomGPT
from config import AGENT_PREFIX, KNOWN_TABLES
from Prompts.few_shot_examples import get_few_shot_examples

from Tools.info_powerbi_tool import InfoPowerBITool
from Tools.list_powerbi_tool import ListPowerBITool
from Tools.schema_powerbi_tool import SchemaPowerBITool
from Tools.query_powerbi_tool import QueryPowerBITool
from Tools.match_query_example_tool import MatchQueryExampleTool

# PowerBI Agent Configuration

In [2]:
def create_powerbi_agent():
    """Create and configure the enhanced PowerBI agent."""

    # Initialize the custom LLM
    llm = CustomGPT()

    # Initialize tools
    tools = [
        InfoPowerBITool(),
        ListPowerBITool(),
        SchemaPowerBITool(),
        QueryPowerBITool(),
        MatchQueryExampleTool()
    ]

    # Convert tools to a format ZeroShotAgent can use
    tool_names = [tool.name for tool in tools]

    # Create a separate string for few-shot examples to avoid template issues
    few_shot_examples = get_few_shot_examples()

    # Define the prefix (system message) for the agent
    prefix = f"""
    {AGENT_PREFIX}

    You are an advanced PowerBI data analyst specializing in complex queries and data analysis. Your expertise lies in understanding 
    star schema architectures with fact and dimension tables, and constructing appropriate joins to answer user questions.

    IMPORTANT WORKFLOW FOR ANSWERING QUESTIONS:

    1. DETERMINE QUESTION TYPE:
       - For simple requests like "list tables" or "show schema", use the appropriate tool directly.
       - For complex analytical queries, first check if there's a matching example.

    2. FOR COMPLEX ANALYTICAL QUERIES:
       a. Use match_query_example_tool to check if there's a similar predefined query
       b. If an EXACT_MATCH is found, use query_powerbi_tool directly with the provided query
       c. If a SIMILAR_MATCH is found, adapt the template query by changing the parameters mentioned in the match result
       d. If NO_MATCH is found, proceed with data exploration and query construction

    3. DATA EXPLORATION (for novel queries or adaptation):
       a. Use info_powerbi_tool to understand the data model
       b. Use schema_powerbi_tool to examine specific tables
       c. Construct an appropriate DAX query based on the user's question
       d. Use query_powerbi_tool to execute the DAX query

    IMPORTANT DAX QUERY RULES:
    1. All DAX queries must start with EVALUATE
    2. Everything after EVALUATE must return a table expression
    3. To return a single value, use ROW() function:
       EVALUATE ROW("Label", CALCULATE([Measure], Filters))
    4. For tabular data, use functions like SUMMARIZECOLUMNS, TOPN, etc.
    
    IMPORTANT DAX SYNTAX RULES:
    1. Never include the language identifier (e.g., "DAX") in the query itself
    2. Always use EVALUATE at the beginning of the query only once
    3. For VAR-based queries, follow this exact pattern:
       EVALUATE
       VAR MyVariable = ...
       RETURN
           MyVariable

    4. Prefer SUMMARIZECOLUMNS over SUMMARIZE for better performance and handling of nulls
    5. Ensure all parentheses are properly balanced
    6. When using action inputs with query_powerbi_tool, format like this:
       Action Input: EVALUATE ... (without extra backticks or language identifiers)

    The most common DAX syntax errors to avoid:
    - Including the language identifier in the query
    - Having multiple EVALUATE statements
    - Forgetting to include a RETURN statement when using VAR
    - Unbalanced parentheses in complex expressions
    
    PREDEFINED MEASURES:
    - [MB]: Marge Brute (Gross Margin) - A calculated measure already defined in the data model
    - [CA]: Chiffre d'Affaires (Revenue) - A calculated measure already defined in the data model

    DAX QUERY PATTERN INSPIRATION:
    - To get a single value (like total revenue):
      EVALUATE ROW("Revenue", CALCULATE([CA], Filters))

    - To get values by time period:
      EVALUATE SUMMARIZECOLUMNS(
          DIM_DATE[Period],
          "Measure", CALCULATE([Measure], Filters)
      )

    - To get top items by some measure:
      EVALUATE TOPN(
          N,
          SUMMARIZECOLUMNS(Table[Column],
          "Measure", CALCULATE([Measure], Filters)),
          [Measure], DESC
      )

    - To compare metrics across dimensions:
      EVALUATE SUMMARIZECOLUMNS(
          Dimension1, Dimension2,
          "Measure1", CALCULATE([Measure1], Filters),
          "Measure2", CALCULATE([Measure2], Filters)
      )

    When constructing queries from scratch, you can be inspired by these patterns
    even if there's no exact match in the predefined examples.

    Always ensure the final query is properly formatted and follows DAX syntax.
    """

    # Escape any problematic characters in the few_shot_examples to prevent template issues
    escaped_examples = few_shot_examples.replace("{", "{{").replace("}", "}}")
    # Restore specific strings that should remain as template variables
    escaped_examples = escaped_examples.replace("{{tool_names}}", "{tool_names}")

    # Combine the prefix, examples, and tools list
    full_prefix = prefix + "\n" + escaped_examples + "\n\nYou have access to the following tools:"

    # Define the agent format instructions with strict guidance on format
    format_instructions = """Use the following format:

    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question

    IMPORTANT EFFICIENCY RULES:
    - For simple requests like "list tables", go directly to the appropriate tool.
    - For complex queries, ALWAYS check match_query_example_tool first before trying to create a query from scratch.
    - If match_query_example_tool returns an EXACT_MATCH, use the provided query directly with query_powerbi_tool.
    - If match_query_example_tool returns a SIMILAR_MATCH, adapt the template query by modifying only the necessary parameters.
    - Only proceed with full data exploration if no matches are found.
    - When writing DAX queries, avoid using CALCULATE directly in EVALUATE. Instead, wrap it in ROW() or use SUMMARIZECOLUMNS.

    IMPORTANT FORMAT RULES:
    - After each "Thought:", you MUST ALWAYS follow with either "Action:" or "Final Answer:".
    - NEVER write a Thought without following it with either an Action or Final Answer.
    - ALWAYS use "Final Answer:" when you have the information needed to answer the user's question.

    EXAMPLE OF CORRECT DAX QUERY FORMAT:
    Action: query_powerbi_tool
    Action Input: 
    EVALUATE
    TOPN(
        3,
        SUMMARIZECOLUMNS(
            MAPPING_PRODUIT[Produit],
            "Total Revenue", CALCULATE(
                SUM(GL[MONTANT]),
                GL[EXERCICE] = 2024
            )
        ),
        [Total Revenue],
        DESC
    )

    Notice there are NO language identifiers, NO extra backticks, and only ONE EVALUATE statement.

    NEVER invent or answer questions the user didn't ask.
    ALWAYS stay focused on the exact question the user is asking.
    """

    # Create the prompt template
    prompt = ZeroShotAgent.create_prompt(
        tools,
        prefix=full_prefix,
        format_instructions=format_instructions,
        input_variables=["input", "chat_history", "agent_scratchpad"]
    )

    # Create the memory
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    # Create the LLM chain
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        llm_chain = LLMChain(llm=llm, prompt=prompt)

    # Create the agent
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)

    # Create the agent executor with increased max_iterations for complex queries
    agent_executor = AgentExecutor.from_agent_and_tools(
        agent=agent,
        tools=tools,
        memory=memory,
        verbose=True,
        max_iterations=8,  # Increased to allow for more complex reasoning chains
        handle_parsing_errors=True,
        early_stopping_method="force",  # Force stopping on parsing errors to avoid infinite loops
    )

    return agent_executor

# PowerBI Agent Initialization

In [3]:
def main():
    # Create the agent
    agent = create_powerbi_agent()

    print("🤖 Your personal PowerBI Agent is initialized and Ready to Go ! ")
    print("📋 Ask questions about your PowerBI dataset or request analysis.")
    print("👋🏽 Type 'exit' to quit.\n")

    while True:
        user_input = input("• Question: ")

        if user_input.lower() == "exit":
            print("Goodbye! 👋🏽")
            break

        try:
            # Let the agent do all the work
            response = agent.invoke({"input": user_input})
            print(f"\nAnswer: {response['output']}\n")
        except Exception as e:
            print(f"Error: {str(e)}")
            print("\nAnswer: I encountered an issue processing your request. Please try rephrasing your question or ask something else.\n")


# 🤖 Agent Capabilities by Complexity Level

The Power BI agent is currently under development and continues to improve. Its ability to answer questions varies based on the complexity of the query. As complexity increases, more training and refinement may be required for optimal performance.

### Level 1: Basic Schema Understanding
**Description**: Straightforward questions about dataset structure.
**Examples**:
- *List all tables in the current dataset.*
- *Provide the schema of the table `GL`.*

### Level 2: Direct Data Retrieval with Explicit References
**Description**: Queries that reference specific tables and column names.
**Examples**:
- *List some `Sous BU` values from the `GL` table.*

### Level 3: Semantic Retrieval with Partial Technical Knowledge
**Description**: Queries that omit exact column names but imply intent.
**Examples**:
- *Which column in `GL` represents revenue? Show some sample values.*
- *List a few products from the `GL` table.*

### Level 4: Natural Language Queries on a Single Table
**Description**: Fully natural language queries referring to a single table without technical terms.
**Example**:
- *What is the total revenue of product `P231` for the year 2024?*

### Level 5: Complex Analytical Queries with Joins
**Description**: Natural language questions requiring data aggregation and joins across multiple tables.
**Example**:
- *Rank the top 3 best-selling products in the `Digital Solutions` Sous BU, where `Rangnar` was the sole client in 2026.*

---

### 🚧 Future Development
The agent's capabilities will expand to better handle high-complexity queries (Level 4 and 5) through improved semantic understanding and more robust training on real-world query examples.


# Testing

In [ ]:
main()

🤖 Your personal PowerBI Agent is initialized and Ready to Go ! 
📋 Ask questions about your PowerBI dataset or request analysis.
👋🏽 Type 'exit' to quit.



• Question:  TOP 3 produits par MB de la sous BU Digital Solutions en 2024




> Entering new AgentExecutor chain...
Thought: This question requires finding the top 3 products by Gross Margin (Marge Brute) for the sub-business unit "Digital Solutions" in the year 2024. I will first use match_query_example_tool to check if there's a predefined query similar to this one.

Action: match_query_example_tool
Action Input: TOP 3 produits par MB de la sous BU Digital Solutions en 2024
Observation: EXACT_MATCH: Found exact match with template question: 'TOP 3 produits par MB de la sous BU Digital Solutions en 2024'
Use this query:

EVALUATE
VAR top3Tavle = 
    TOPN(
        3,
        SUMMARIZE(
            MAPPING_PRODUIT,
            MAPPING_PRODUIT[Produit],
            "Marge Brute", CALCULATE(
                [MB],
                GL[Sous BU] = "Digital Solutions",
                DIM_DATE[Année] = 2024
            )
        ),
        [Marge Brute],
        DESC
    )
VAR TOP3TRIEE = ADDCOLUMNS(
    top3Tavle,
    "Rank Marge Brute", RANKX(
        top3Tavle,
   

• Question:  Give me the four best selling products in terms of revenu, for the year 2023




> Entering new AgentExecutor chain...
Thought: I need to find the top four best selling products in terms of revenue for the year 2023. I'll start by checking if there's a predefined query example that matches this question.

Action: match_query_example_tool
Action Input: Give me the four best selling products in terms of revenue, for the year 2023
Observation: SIMILAR_MATCH: Found similar match with template question: 'give me the best selling three products in termes of revenu, in the year 2024'
Similarity score: 69%
Parameters that may need adaptation:
- Year: from 2024 to 2023

Template query:

EVALUATE
TOPN(
    3,
    SUMMARIZECOLUMNS(
        MAPPING_PRODUIT[Produit],
        "Total Revenue", 
        CALCULATE(
            SUM(GL[MONTANT]),
            GL[EXERCICE] = 2024
        )
    ),
    [Total Revenue],
    DESC
)
I found a similar query for getting the best-selling three products in terms of revenue, but it is for the year 2024. I need to adapt this query to get the to

• Question:  Quelle est l'evolution du chiffre d'affaire mensuel sur T1 et T2 2024, du client Annah de la sous BU Back Office




> Entering new AgentExecutor chain...
This question requires analyzing the monthly revenue evolution for the client "Annah" within the sub-business unit "Back Office" for the first two quarters (T1 and T2) of 2024.

Observation: Invalid Format: Missing 'Action:' after 'Thought:
Action: match_query_example_tool
Action Input: evolution du CA mensuel sur T1 et T2 2024 du client Annah de la sous BU Back Office
Observation: SIMILAR_MATCH: Found similar match with template question: 'Evolution du CA mensuel sur T1 et T2 2024 du client Anah de la sous BU Back Office'
Similarity score: 94%

Template query:

EVALUATE
SUMMARIZECOLUMNS(
    DIM_DATE[Mois],
    DIM_DATE[MOIS_NOM],
    "Chiffre d'Affaires", 
    CALCULATE(
        [CA],
        DIM_CLIENT[RAISON_SOCIALE_DO] = "Annah",
        GL[Sous BU] = "Back office",
        DIM_DATE[Année] = 2024,
        DIM_DATE[Mois] >= 1,
        DIM_DATE[Mois] <= 6
    )
)
ORDER BY DIM_DATE[Mois]
The provided query template is very similar to the user's

Sample of questions : 

Level 1 : List me all the table in the current dataset

Level 3 : Give me 10 products from the table GL

Level 4/5 : Quelle est l'evolution du chiffre d'affaire mensuel sur T1 et T2 2024, du client Annah de la sous BU Back Office

Level 4/5 : TOP 3 produits par MB de la sous BU Digital Solutions en 2024

Level 4/5 : Give me the four best selling products in terms of revenu, for the year 2023